In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from qiskit import QuantumRegister, QuantumCircuit
from qiskit import Aer
from qiskit.visualization import plot_state_city
import qiskit.quantum_info as qi

from qelvin import QRegister, QCircuit

# Quantum simulation of Schrödinger's equation

Consider a single particle in one-dimensional space with Hamiltonian

$$\begin{equation}
H=\frac{p^2}{2m}+V(x),
\end{equation}$$

where $p$ is the momentum operator and $x$ is the position operator. The Schrödinger equation for the Hamiltonian is

$$\begin{equation}
ih\dfrac{\partial}{\partial t}|\psi\rangle=(-\frac{1}{2m}\dfrac{\partial^2}{\partial x^2}+V(x))|\psi\rangle,
\end{equation}$$

where $|\psi\rangle$ is the system state that resides in an infinite dimensional Hilber space. In the $x$ basis, it can be written as

$$\begin{equation}
|\psi\rangle=\int_{-\infty}^\infty|x\rangle\langle x|\psi\rangle dx.
\end{equation}$$

Let us define the potential to be

$$\begin{equation}
V(x)=\begin{cases}
0, & -L\leq x\leq L\\
\infty, & \mathrm{elsewhere}
\end{cases}.
\end{equation}$$

By choosing a differential step size $\Delta x$ the state can be described with equation

$$\begin{equation}
|\tilde{\psi}\rangle=\sum_{k=-\frac{L}{\Delta x}}^{\frac{L}{\Delta x}}a_k|k\Delta x\rangle.
\end{equation}$$

This state can be represented with $n=\lceil\log{\left(\frac{2L}{\Delta x}+1\right)}\rceil$ qubits. The time evolution of the system over time $t$ is defined by equation

$$\begin{equation}
|\tilde{\psi}(t)\rangle=e^{-iHt}|\tilde{\psi}(0)\rangle.
\end{equation}$$

The time evolution of a single state over time step $\Delta t$ can be described with

$$\begin{equation}
|k\rangle\rightarrow e^{-i\frac{p^2}{2m}}e^{-iV(k\Delta x)\Delta t}|k\rangle.
\end{equation}$$

Since $x=k\Delta x$ and $p$ are conjugate variables related by quantum Fourier transform $U_\mathrm{FFT}xU_\mathrm{FFT}^\dagger=p$ we get
$$\begin{equation}
|k\rangle\rightarrow U_\mathrm{FFT}e^{-i\frac{k^2(\Delta x)^2}{2m}}U_\mathrm{FFT}^\dagger e^{-iV(k\Delta x)\Delta t}|k\rangle.
\end{equation}$$


### Computable phase shifts

Let $m$ and $n$ be positive integers and $f:\left\{0,...,2^m-1\right\}\rightarrow\left\{0,...,2^n-1\right\}$ a classcal function from $m$ to $n$ bits which may be computed reversibly using $T$ Toffoli gates. Let $\hat{P}$ be the an operator which can be constructed with $T$ Toffoli gates and for which

$$\begin{equation}
\hat{P}|x,y\rangle\rightarrow|x,y\oplus f(x)\rangle.
\end{equation}$$

Consider state $|x, q\rangle$ where $q=\sum_{j=0}^{2^n-1}q_j2^j$ and $q_j\in\left\{0,1\right\}$. Let us define operator

$$\begin{equation}
\hat{O}|x,q\rangle=e^\frac{2\pi iyn}{2^n}\prod_{j=0}^{n-1}e^{-\pi iq_j2^{j-n+1}}|x, q\rangle=e^{-\frac{2\pi i}{2^n}(\sum_{j=0}^{2^n-1}q_j2^j-y)}|x, q\rangle=e^{-\frac{2\pi i}{2^n}(q-y)}|x,q\rangle.
\end{equation}$$

We now obtain

$$\begin{equation}
\hat{O}|x,y\oplus f(x)\rangle=e^{-\frac{2\pi i}{2^n}f(x)}|x,y\oplus f(x)\rangle,
\end{equation}$$

and thus trasformation

$$\begin{equation}
|x\rangle\rightarrow e^{-\frac{2\pi i}{2^n}f(x)}|x\rangle
\end{equation}$$

can be computed with operation $\hat{P}\hat{O}\hat{P}^\dagger$. $\prod_{j=0}^{n-1}e^{-\pi iq_j2^{j-n+1}}$ can be expressed with $n$ single qubit phase gates $\hat{U}_{n-1}\cdot\cdot\cdot\hat{U}_0$ and $e^\frac{2\pi iyn}{2^n}$ can be ignored since the end result does not depend on $y$. We obtain

$$\begin{equation}
\hat{O}=\hat{U}_{n-1}\cdot\cdot\cdot\hat{U}_0,
\end{equation}$$
where $\hat{U}_j=\begin{pmatrix}1 & 0 \\ 0 & e^{-\pi i2^{j-n+1}}\end{pmatrix}$.

### QFT multiplier

In [21]:
dt = 0.1
dx = 0.03
L = 0.1
m = 1.0

def V(x):
    return 0;

def toffoli(circ, a, b, c):
    theta = np.pi/4.0
    circ.h(c)
    circ.cx(b, c)
    circ.p(-theta, c)
    circ.cx(a, c)
    circ.p(theta, c)
    circ.cx(b, c)
    circ.p(-theta, c)
    circ.cx(a, c)
    circ.p(-theta, b)
    circ.p(theta, c)
    circ.cx(a, b)
    circ.h(c)
    circ.p(-theta, b)
    circ.cx(a, b)
    circ.p(theta, a)
    circ.p(np.pi/2.0, b)

def swap(circ, a, b):
    circ.cx(a, b)
    circ.cx(b, a)
    circ.cx(a, b)

N = int(np.ceil(np.log(2*L/dx+1)))

psi_input = QRegister(3, 0)
print(psi_input)

circ = QCircuit(psi_input)

for j in range(N):
    for k in range (j):
        circ.cp(np.pi/float(2**(j-k)), j, k)
    circ.h(j)

for j in range(int(np.floor(N/2))):
    swap(circ, j, N-j-1)

print(circ)
circ.run()

for j in range(int(np.floor(N/2))):
    swap(circ, j, N-j-1)

for j in reversed(range(N)):
    circ.h(j)
    for k in reversed(range(j)):
        circ.cp(np.pi/float(2**(j-k)), j, k)

print(circ)
circ.run()

psi_output = circ.state()
print(psi_output)


[ 1.000+0.000*j  0.000+0.000*j  0.000+0.000*j  0.000+0.000*j  0.000+0.000*j  0.000+0.000*j  0.000+0.000*j  0.000+0.000*j ]
      ---------  ---------             ---------                                   ---------                
     |         ||         |           |         |                                 |         |               
q0---|    H    ||P(+1.571)|-----------|P(+0.785)|---------------------------o-----|    X    |-----o---------
     |         ||         |           |         |                           |     |         |     |         
      ---------  ---------             ---------                            |      ---------      |         
                     |      ---------      |      ---------                 |          |          |         
                     |     |         |     |     |         |                |          |          |         
q1-------------------o-----|    H    |-----|-----|P(+1.571)|----------------|----------|----------|---------
     